# 5W

In [1107]:
import activityinfo as AT
import pandas as pd
import numpy as np
import geopandas as gpd
import time

In [1108]:
## definimos variables para el calculo mensual
mes_curso = 'AI2022-02.xlsx'
mes = '2022-02'

In [1109]:
df_activity_info = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\DescargasMensuales\procesados\proc_'+mes_curso, index_col=0)

# Tablas de referencia

In [1110]:
# Merge para appealing organization
Organizations_ref = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='ImplementingPartners')
Organizations_ref = Organizations_ref[['Nombre_ecu', 'Name']]
Organizations_ref = Organizations_ref[Organizations_ref['Nombre_ecu'].notna()]

In [1111]:
# Sectores
sectores_ref= pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='sectores')

In [1112]:
MarcoMonitoreo_ref= pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='marco_monitoreo')


In [1113]:
delivery_mechanism = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='delivery_mech')

In [1114]:
df_activity_info = df_activity_info[df_activity_info['Database'].notna()]
df_activity_info['Country'] = 'Ecuador'
df_activity_info['Country Admin1'] = df_activity_info['DPA_DESPRO'].str.title()
df_activity_info['Admin2'] = df_activity_info['DPA_DESCAN'].str.title()
df_activity_info = df_activity_info[df_activity_info['mes']==mes]

In [1115]:
df_activity_info = pd.merge(df_activity_info, Organizations_ref, how='left', left_on='donante.nombre',
                                    right_on='Nombre_ecu')
df_activity_info.drop(['Nombre_ecu'], axis=1, inplace=True)
df_activity_info.rename(columns = {'Name':'Appealing organisation Name'}, inplace = True)

In [1116]:
# Implementation set up
df_activity_info['Implementation Set up'] = np.where(df_activity_info['implementación'] == 'Directa',
                                                 'No',
                                                 'Yes')

In [1117]:
df_activity_info = pd.merge(df_activity_info, Organizations_ref, how='left', left_on='implementador.nombre',
                                    right_on='Nombre_ecu')
df_activity_info.drop(['Nombre_ecu'], axis=1, inplace=True)
df_activity_info.rename(columns = {'Name':'Implementing partner Name'}, inplace = True)

In [1118]:
# Month
df_activity_info['Month'] = df_activity_info['mes']


In [1119]:
# Subsector
df_activity_info = pd.merge(df_activity_info, sectores_ref, how='left', left_on='sector',
                                    right_on='sector')

In [1120]:
# Indicador
df_activity_info['Indicator'] = df_activity_info['indicador_eng']
df_activity_info = df_activity_info[df_activity_info['Indicator'].notna()]


In [1121]:
#Activity
df_activity_info['Activity Name'] = df_activity_info['actividad']

In [1122]:
#Activity Description
df_activity_info['Activity Description'] = ''

In [1123]:
#COVID 19 Situation
df_activity_info['COVID 19 Situation'] = df_activity_info['¿esta_actividad_se_realiza_en_el_marco_de_respuesta_a_la_emergencia_sanitaria_por_el_covid-19?']
df_activity_info['COVID 19 Situation'] = np.where(df_activity_info['COVID 19 Situation'] == 'Si',
                                                 'Yes',
                                                 'No')


In [1124]:
#RMRP Activity
df_activity_info['RMRP Activity'] = df_activity_info['¿esta_actividad_ha_sido_planificada_en_el_rmrp_2022?']
df_activity_info['RMRP Activity'] = np.where(df_activity_info['RMRP Activity'] == 'Si',
                                                 'Yes',
                                                 'No')

In [1125]:
#CVA
df_activity_info['CVA'] = np.where(df_activity_info['CVA_tag'] == 'Si',
                                                 'Yes',
                                                 'No')


In [1126]:
# Value (in USD)
USD_records = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Monto transferido'],
                                                       aggfunc='max',
                                                       fill_value=0,
                                                       margins='False')
df_activity_info = pd.merge(df_activity_info, USD_records, how='left', left_on='RecordId',
                                    right_on='RecordId')
df_activity_info.rename(columns = {'Monto transferido_y':'Value (in USD)'}, inplace = True)
df_activity_info['Value (in USD)'] = df_activity_info['Value (in USD)'].replace(np.nan, 0)
                                                

In [1127]:
#Delivery mechanism

df_activity_info = pd.merge(df_activity_info, delivery_mechanism, how='left', left_on='mecanismo_de_entrega',
                                    right_on='mecanismo_de_entrega')
df_activity_info['Delivery mechanism'] = df_activity_info['Delivery mechanism'].replace(np.nan, '')

df_5w_part = df_activity_info[["RecordId","Country","Country Admin1","Admin2","Appealing organisation Name","Implementation Set up","Implementing partner Name","Month","Subsector","Indicator","Activity Name","Activity Description","COVID 19 Situation","RMRP Activity","CVA","Value (in USD)","Delivery mechanism"]]

df_5w_part.drop_duplicates(keep='first', inplace=True)

<ipython-input-1127-e502d9900886>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5w_part.drop_duplicates(keep='first', inplace=True)


In [1128]:
df_activity_info['aux']= df_activity_info['población meta'] + "-" + df_activity_info['frecuencia'] + "-" + df_activity_info['unidad_combinada']

In [1129]:
df_activity_info['aux'] = df_activity_info['aux'].fillna('Quantity of output')
auxPivot = df_activity_info.pivot(columns=['aux'], values='Value').fillna(0)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)


In [1130]:
#Quantity of output
Quantity_of_output = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Quantity of output'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, Quantity_of_output, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [1131]:
# Total monthly beneficiaries
auxPivot=auxPivot[auxPivot.columns.drop(list(auxPivot.filter(regex='# de personas')))]
auxPivot=auxPivot[auxPivot.columns.drop('Quantity of output')]
auxPivot['Total monthly beneficiaries']=auxPivot.sum(axis = 1, skipna = True)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
total_monthly = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Total monthly beneficiaries'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, total_monthly, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [1132]:
#New beneficiaries of the month
nuevos=list(auxPivot.filter(regex='Nuevo'))
auxPivot['New beneficiaries of the month']= auxPivot[nuevos].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
new_beneficiaries_month = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['New beneficiaries of the month'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, new_beneficiaries_month, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [1133]:
#Refugees and Migrants IN DESTINATION
refugiados_migrantes=list(auxPivot.filter(regex='Refugiados y migrantes'))
auxPivot['Refugees and Migrants IN DESTINATION']= auxPivot[refugiados_migrantes].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
rrmm_in_destination = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Refugees and Migrants IN DESTINATION'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, rrmm_in_destination, how='left', left_on='RecordId',
                                    right_on='RecordId')


In [1134]:
#Refugees and Migrants IN TRANSIT
df_5w_part['Refugees and Migrants IN TRANSIT'] = 0

In [ ]:
#Host Communities Beneficiaries
comunidad_acogida=list(auxPivot.filter(regex='Comunidad de acogida'))
auxPivot['Host Communities Beneficiaries']= auxPivot[comunidad_acogida].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
ccaa_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Host Communities Beneficiaries'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, ccaa_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [ ]:
#Refugees and Migrants PENDULARS
df_5w_part['Refugees and Migrants PENDULARS'] = 0
df_5w_part['Colombian Returnees'] = 0

In [ ]:
# Women under 18
ninas=list(auxPivot.filter(regex='Niñas'))
auxPivot['Women under 18']= auxPivot[ninas].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
ninas_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Women under 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, ninas_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [ ]:
# Men under 18
ninos=list(auxPivot.filter(regex='Niños'))
auxPivot['Men under 18']= auxPivot[ninos].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
ninos_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Men under 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, ninos_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [ ]:
#Women above 18
mujeres=list(auxPivot.filter(regex='Mujeres'))
auxPivot['Women above 18']= auxPivot[mujeres].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
mujeres_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Women above 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, mujeres_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [ ]:
#Men above 18
hombres=list(auxPivot.filter(regex='Hombres'))
auxPivot['Men above 18']= auxPivot[hombres].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
hombres_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Men above 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, hombres_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [ ]:
# Other under 18
df_5w_part['Other under 18'] = 0

In [ ]:
#Other above 18
otros=list(auxPivot.filter(regex='Otros'))
auxPivot['Other above 18']= auxPivot[otros].sum(axis=1)
df_activity_info = pd.merge(df_activity_info, auxPivot, left_index=True, right_index=True)
otros_beneficiaries = df_activity_info.pivot_table(index=['RecordId'],
                                                       values=['Other above 18'],
                                                       aggfunc='sum',
                                                       fill_value=0,
                                                       margins='False')
df_5w_part = pd.merge(df_5w_part, otros_beneficiaries, how='left', left_on='RecordId',
                                    right_on='RecordId')

In [ ]:
#platform
df_5w_part["Platform"] = df_5w_part["Country"]

In [ ]:
df_activity_info.to_clipboard(excel=True)

In [ ]:
# final change appealing vs implementing

df_5w_part['Appealing organisation Name'] = np.where(df_5w_part['Implementation Set up'] == 'No',
                                                 df_5w_part['Implementing partner Name'],
                                                 df_5w_part['Appealing organisation Name'])

In [ ]:
df_5w_part['Implementing partner Name'] = np.where(df_5w_part['Implementation Set up'] == 'No',
                                                 '',
                                                 df_5w_part['Implementing partner Name'])

In [ ]:
df_5w_part.to_clipboard(excel=True)